#### Import

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import tensorflow.keras.preprocessing.image as p

#### 데이터불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# test_data =pd.read_csv('./test.csv')
# train_data = pd.read_csv('./train.csv')
# data = pd.read_csv('./sub+1.csv')  # 전처리 끝난 데이터

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/오일공모전/train_v1.csv')

In [ ]:
data

,Unnamed: 0,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,0,TRAIN_00000,COMPONENT3,1486.0,2011.0,7.0,200.0,0.0,3.0,93.0,...,0.0,0.0,0.0,0.0,0.0,0.0,13.9,154.0,75.0,0
1,1,TRAIN_00001,COMPONENT2,1350.0,2021.0,51.0,375.0,0.0,2.0,19.0,...,2.0,4.0,6.0,216.0,1454.0,0.0,12.9,44.0,652.0,0
2,2,TRAIN_00002,COMPONENT2,2415.0,2015.0,2.0,200.0,0.0,110.0,1.0,...,0.0,3.0,39.0,11261.0,41081.0,0.0,13.3,72.6,412.0,1
3,3,TRAIN_00003,COMPONENT3,7389.0,2010.0,2.0,200.0,0.0,8.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,133.3,7.0,0
4,4,TRAIN_00004,COMPONENT3,3954.0,2015.0,4.0,200.0,0.0,1.0,157.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.1,133.1,128.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,14090,TRAIN_14090,COMPONENT3,1616.0,2014.0,8.0,200.0,0.0,2.0,201.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,135.4,16.0,0
14091,14091,TRAIN_14091,COMPONENT1,2784.0,2013.0,2.0,200.0,0.0,3.0,85.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.5,117.5,1408.0,0
14092,14092,TRAIN_14092,COMPONENT3,1788.0,2008.0,9.0,550.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,13.6,54.0,1301.0,0
14093,14093,TRAIN_14093,COMPONENT2,2498.0,2009.0,19.0,550.0,0.0,2.0,4.0,...,7.0,8.0,100.0,1625.0,18890.0,0.0,13.2,44.3,652.0,0


#### YEAR 포함 필요한 컬럼만 모으기

In [ ]:
x_train_data = pd.DataFrame(data) 
x_train_data = data.loc[:,['COMPONENT_ARBITRARY','ANONYMOUS_1','YEAR','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
y_data = pd.DataFrame(data['Y_LABEL'])

In [ ]:
y_data = pd.get_dummies(data['Y_LABEL'])

#### 전처리

In [ ]:
# COMPONENT_ARBITRARY 라벨인코딩
le =LabelEncoder()
x_train_data_le = x_train_data

In [ ]:
# COMPONENT_ARBITRARY 라벨인코딩 후 합치기
x_train_data_le["COMPONENT_ARBITRARY"] = le.fit_transform(x_train_data["COMPONENT_ARBITRARY"])

In [ ]:
# 로부스트스케일링
robustScaler = RobustScaler()
train_data_robustScaled = robustScaler.fit_transform(x_train_data_le)
train_data_robustScaled

array([[ 0.        , -0.34593838, -0.42857143, ...,  0.        ,
         0.65443425, -0.41127542],
       [-0.5       , -0.40943044,  1.        , ...,  0.        ,
        -1.02752294,  0.1219963 ],
       [-0.5       ,  0.08776844,  0.14285714, ...,  0.        ,
        -0.59021407, -0.09981516],
       ...,
       [ 0.        , -0.20494865, -0.85714286, ...,  0.        ,
        -0.87461774,  0.72181146],
       [-0.5       ,  0.12651727, -0.71428571, ...,  0.        ,
        -1.02293578,  0.1219963 ],
       [-0.5       , -0.15172736, -0.28571429, ...,  0.        ,
        -0.98165138,  0.08502773]])

Call back /// Learning Rage 조절하는 함수

In [ ]:
annealer = ReduceLROnPlateau(monitor = 'loss', factor = 0.5, patience = 10, verbose = 1, min_lr = 1e-4 )
# https://deep-deep-deep.tistory.com/56
MODEL_SAVE_FOLDER_PATH='C:/Users/NTX550/Desktop/건설기계오일분류/' #데이터 저장
model_path = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_loss:.4f}.hdf5'## epoch 저장
checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_accuracy', verbose = 1, save_best_only = True)
earlystop = EarlyStopping(patience = 20)
x_data_callbacks = [earlystop, checkpoint, annealer]

Dense Layer

In [ ]:
model = Sequential()
model.add(Dense( units = 512,  kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense( units = 256, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense( units = 128, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense( units = 64, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense( units = 2, kernel_initializer = 'uniform', activation = 'softmax'))
# from keras.layers import BatchNormalization (배치정규화)
#model.add(BatchNormalization())

첫번째(units = 출력뉴런 수), # input_dim(입력 뉴런의 수)

 
kernel_initializer = 가중치 초기화 하는법 ( uniform : 균일 분포, normal : 가우시안 분포)


activation = 활성함수 설정 (relu : 은닉층에서 주로 쓰임, sigmoid : 이진분류, softmax : 다중클래스)

In [ ]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics=['acc'])
model.fit(train_data_robustScaled,y_data, epochs = 10)

#shape를 해줘야 될것같은데... 아니였다 Y값을 안넣어서 그랬음 Y값 넣어줄때 onehot인코딩 해줘야함 ()
# pd.get_dummies(data['Y_LABEL']) 로 EZ하게 가능

compile = keras(케라스)모델의 layers 컴파일 해줘야함

optimizer = 훈련 과정을 설정하는 옵티마이저를 설정합니다.

metrics = 훈련 모니터링하기 위한 지표

loss = 훈련시 사용할 손실함수 선택(loss function)
회귀 문제	mean_squared_error	-	선형 회귀 실습
다중 클래스 분류	categorical_crossentropy	소프트맥스	로이터 뉴스 분류하기
다중 클래스 분류	sparse_categorical_crossentropy	소프트맥스	양방향 LSTM를 이용한 품사 태깅
이진 분류	binary_crossentropy	시그모이드	IMDB 리뷰 감성 분류하기

In [ ]:
test = pd.read_csv("/content/test.csv")
test

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,TEST_0000,COMPONENT1,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
1,TEST_0001,COMPONENT3,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12
2,TEST_0002,COMPONENT2,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714
3,TEST_0003,COMPONENT3,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94
4,TEST_0004,COMPONENT2,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,COMPONENT3,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163
6037,TEST_6037,COMPONENT3,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13
6038,TEST_6038,COMPONENT3,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13
6039,TEST_6039,COMPONENT2,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478


In [ ]:
test.drop(columns=["ID"], inplace=True)
test["COMPONENT_ARBITRARY"] = le.transform(test["COMPONENT_ARBITRARY"])
tset_data = robustScaler.transform(test)

In [ ]:
pred = model.predict(tset_data).argmax(axis=1)
pred

189/189 [==============================] - 1s 2ms/step


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
submit = pd.DataFrame()
submit["ID"] = test["ID"]
submit["Y_LABEL"] = pred
submit

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
6036,TEST_6036,0
6037,TEST_6037,0
6038,TEST_6038,0
6039,TEST_6039,0


In [ ]:
submit.to_csv("/content/submit_test.csv", index=False)